# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [156]:
# import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as st

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [157]:
# Your code here:

salaries = pd.read_csv('./Current_Employee_Names__Salaries__and_Position_Titles.csv')



Examine the `salaries` dataset using the `head` function below.

In [158]:
# Your code here:

#salaries.reset_index(inplace=True)

salaries.sample(5)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
22282,"PAGAN, LAURA M",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN
22242,"PACINO, WENDY A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
76,"ACEVEDO, JASON M",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN
29060,"TALAVERA, FEDERICO",FIREFIGHTER-EMT,FIRE,F,Salary,NaN,92274.0,NaN
18662,"MC CLURE, JAMES E",FIREFIGHTER,FIRE,F,Salary,NaN,87006.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [159]:
# Your code here:

salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [160]:
# Your code here:

salaries['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [161]:
# Your code here:
salaries['Department'].value_counts()


POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [162]:
# Your code here:

#1. set the hypothesis
#h0: hourly wage = $30/hr
#h1: hourly wage ! = $30/hr
#two sided direction

#2 choose significance level: 95%
sig_level=0.05

#3 sample
hourly_sample = salaries[salaries['Salary or Hourly']=='Hourly'].sample(50)



In [163]:
#4 compute statistic
#sample mean - pop mean/(pop stdev/np.sqrt(n)
#however, we don't have pop stdev(replicating industry scenario) so we use s (sample stdev) and t distribution with
#n-1 degrees of freedom
#sample mean - pop mean/(sample stdev/np.sqrt(n)
n=50
mu = 30
mean = hourly_sample['Hourly Rate'].mean()
s = hourly_sample['Hourly Rate'].std(ddof=1)

statistic = (mean - mu)/(s/np.sqrt(n))
print(statistic)
#note my sample mean is 1.68 stdevs from the H0 mean (t-score?)

#5 get p-value (how unlikely it is to be 1.68 stdevs from the mean 

p_value = st.t.sf(abs(statistic),n-1)*2

print(p_value)

#6 decide

#sig level = 1-confidence level, so 0.05 error is acceptable, however I have ~0.098 - H0 still stands

1.6857555358212528
0.09820160127401995


In [164]:
#using built in function 

st.ttest_1samp(hourly_sample['Hourly Rate'],mu)

Ttest_1sampResult(statistic=1.6857555358212528, pvalue=0.09820160127401995)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [165]:
# Your code here:

#1. set hypothesis h0: ave salary > 86k/yr for all salaried employees
#note one sided

#2. sig level = 5%

#3. sample
police_sample = salaries[salaries['Department']=='POLICE']['Annual Salary'].sample(50)


In [166]:
#4 compute statistic
# sample mean - popmean/(sample std/np.sqrt(n)) using t distribution with n-1 degrees of freedom

mu = 86000
mean = police_sample.mean()
s = police_sample.std(ddof=1)
n=50

statistic = (mean - mu)/(s/np.sqrt(n))
print(statistic)

#5. get p-value (don't times by 2)

p_value = st.t.sf(abs(statistic),n-1)
p_value

#6 - first of all the statistic has a negative value then secondary it has a 46% error, H0 cannot be rejected

-0.0991302404662197


0.4607196529908868

In [167]:
#using inbuilt function

st.ttest_1samp(police_sample,mu)[1]/2

0.4607196529908868

Using the `crosstab` function, find the department that has the most hourly workers. 

In [168]:
# Your code here:

cross_tab = pd.crosstab(index = salaries['Department'],columns=salaries['Salary or Hourly'])
cross_tab[cross_tab['Hourly']== cross_tab['Hourly'].max()]

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [171]:
# Your code here:
#1.Set hypothesis: H0: mu <= 35/hr

#2. sig level: 0.05
sig_level = 0.05

#3. select sample

streetsan_sample = salaries[(salaries['Department']=='STREETS & SAN') & (salaries['Salary or Hourly']=='Hourly')]['Hourly Rate'].sample(50)






In [172]:
#4 compute statistic & #5 compute p-value
mu=35
print(st.ttest_1samp(streetsan_sample,mu))
print(st.ttest_1samp(streetsan_sample,mu)[1]/2)

#6 decide
st.ttest_1samp(streetsan_sample,mu)[1]/2 <= sig_level

#True so H0 cannot be rejected, the street&san workers hypothesis holds. 

Ttest_1sampResult(statistic=-2.308330704889571, pvalue=0.025242075340384255)
0.012621037670192127


True

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [178]:
# Your code here:
#we need sample mean, number of observations -1, standard error


In [189]:
#1. confidence level
cl=0.95

#2 sample, mean, std, t-score, n by hand
hourly_wages = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate']
mean = hourly_wages.mean()
s = hourly_wages.std(ddof=1)
n=len(hourly_wages)-1
t_score = st.t.ppf(1-((1-0.95))/2,n-1)

print('left end: ',mean - t_score *(s/np.sqrt(n)))
print('right end: ',mean + t_score *(s/np.sqrt(n)))

#3 using function

print(st.t.interval(0.95,loc=mean,scale=s/np.sqrt(n),df=n-1))



left end:  32.52344181507858
right end:  33.0536736174819
(32.52344181507858, 33.0536736174819)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [207]:
# Your code here:
#1. confidence level
cl=0.95

#2 sample and necessary statistics
police_salaries = salaries[salaries['Department']=='POLICE']['Annual Salary']
mean = police_salaries.mean()
n = len(police_salaries)
s = police_salaries.std(ddof=1)
t_score = st.t.ppf(1-((1-cl))/2,n-1)
print(t_score)
print('left end: ', mean - t_score*(s/np.sqrt(n)))
print('right end: ',mean + t_score*(s/np.sqrt(n)))

#3 using function
st.t.interval(cl,loc=mean,scale=s/np.sqrt(n),df=n-1)


1.9601408638119977
left end:  86177.17166932367
right end:  86795.65733694311


(86177.17166932367, 86795.65733694311)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [283]:
# so if normally distributed, mean = p(proportion)
#std is np.sqrt(p(1-p)/n)
#as long as np >= 10 and n(1-p) >=10

#1. H0: % of hourly workers in Chi town = 25%,H1: % of hourly workers in Chi town != 25%
# 2 sided

#2. significance level
sig_level = 0.05

#3. sample
sample=salaries[salaries['Salary or Hourly']=='Hourly']

#4. compute the statistic

n = len(salaries[salaries['Salary or Hourly']=='Hourly'])
p=0.25
actual_p=len(salaries[salaries['Salary or Hourly']=='Hourly'])/len(salaries)
print(len(salaries[salaries['Salary or Hourly']=='Hourly']))
print(len(salaries))

#z test statistic = (p-p0) / √p0(1-p0)/n
z = (actual_p-p)/np.sqrt(p*(1-p)/n)
print(z)

#5. calculate p-level

p_value = st.norm.sf(abs(z))*2
print(p_value)

#6. decide

p_value > sig_level

#as this is true, the null hypothesis cannot be rejected. 


8022
33183
-1.7063934428910539
0.08793485063288027


True

In [282]:
#using inbuilt function, however David thinks I am doing something wrong here, he trusts my by hand method above.


import statsmodels as sm
from statsmodels import stats
from statsmodels.stats import weightstats
from statsmodels.stats import proportion

sm.stats.proportion.proportions_ztest(8022,33183,value=0.25,alternative='two-sided')

(-3.5099964213703005, 0.0004481127249057967)